In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
cd "/content/drive/MyDrive/fonts classifiction"

/content/drive/MyDrive/fonts classifiction


In [8]:
# -*- coding: utf-8 -*-
"""
Created on Sat Jan  1 21:29:28 2022

@author: David BM

At this file we preprocessing the data: we take the images at the data
and for each letter in the image we generate seperate image and
save her and her belong font in h5 file.

than we augment the images by adding noise and blur
to each corpped image. By this method we multiply our
data set by 3.

"""

import logging
import h5py
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import math
from PIL import Image
from PIL import ImageFilter

In [9]:
def noise_image(pil_im):
    # Adding Noise to image
    img_array = np.asarray(pil_im)
    mean = 0.0   # some constant
    std = 5   # some constant (standard deviation)
    noisy_img = img_array + np.random.normal(mean, std, img_array.shape)
    noisy_img_clipped = np.clip(noisy_img, 0, 255)
    noise_img = Image.fromarray(np.uint8(noisy_img_clipped)) # output
    noise_img=noise_img.resize((32,32))
    return noise_img

def blur_image(pil_im):
    #Adding Blur to image
    blur_img = pil_im.filter(ImageFilter.GaussianBlur(radius=1)) # ouput
    #imshow(blur_img)
    blur_img=blur_img.resize((32,32))
    return blur_img

In [10]:
db=h5py.File('SynthText.h5', 'r')
im_names= list(db['data'].keys())
# there are 973 images
# therefore len(im_names)=973

f=h5py.File('adaptedImage.h5','a')
# f is an output file for him: each image represent only one letter


In [12]:

'''

#preprocessing the data

'''

cnt=0;
for j in range(len(im_names)):

    im=im_names[j]
    img=db['data'][im][:]

    font=db['data'][im].attrs['font']
    txt=db['data'][im].attrs['txt']
    charBB=db['data'][im].attrs['charBB']
    # charBB.shape is (2,4,number of characthers in the image)
    # '2' is standing for two dimionsinal (x,y)
    # '4' is standing for '4' corner point in the chosen dim
    wordBB=db['data'][im].attrs['wordBB']
    # wordBB.shape is (2,4,number of words)

    #extract the characthers from the image

    maxHeight=0
    maxwidth=0
    numOfchars=-1;
    for word in txt:
        for c in word:
            #c=characther in ASCI code
            cnt=cnt+1;
            numOfchars=numOfchars+1;

            # crop the imgae by 4 points of the rectangle of the char
            x_cord=list()
            y_cord=list()
            for i in range(4):
                x=math.trunc(charBB[0][i][numOfchars])
                x_cord.append(x)
                y=math.trunc(charBB[1][i][numOfchars])
                y_cord.append(y)

            left=min(x_cord)
            right=max(x_cord)
            lower=max(y_cord)
            upper=min(y_cord);
            box=(left,upper,right,lower)
            count=0
            #try:
            PIL_image = Image.fromarray(img.astype('uint8'), 'RGB')
            crop_img =PIL_image.crop(box)

            newsize=(32,32)
            crop_img = crop_img.resize(newsize)
            crop_img = np.array(crop_img)

            #enter the image-char to the new dataset
            Myname=im+ " " + str(cnt)
            letter_image=f.create_dataset(Myname,data=crop_img)
            letter_image.attrs['font']=str(font[numOfchars])
            letter_image.attrs['letter']=c

            #seting the base from the label value
            if(letter_image.attrs['font']=="b'Alex Brush'"):
                label=0
            elif(letter_image.attrs['font']=="b'Michroma'"):
                label=1;
            elif(letter_image.attrs['font']=="b'Raleway'"):
                label=2;
            elif(letter_image.attrs['font']=="b'Russo One'"):
                label=3;
            elif(letter_image.attrs['font']=="b'Open Sans'"):
                label=4;
            elif(letter_image.attrs['font']=="b'Ubuntu Mono'"):
                label=5;
            elif(letter_image.attrs['font']=="b'Roboto'"):
                label=6;
            letter_image.attrs['label']=label

            #agumentation of the images
            pil_im=Image.fromarray(img.astype('uint8'), 'RGB')

            noise=noise_image(pil_im)
            cnt=cnt+1
            Myname=im+ " " + str(cnt)
            noise_img=f.create_dataset(Myname,data=noise)
            noise_img.attrs['font']=letter_image.attrs['font']
            noise_img.attrs['letter']=letter_image.attrs['letter']
            noise_img.attrs['label']=letter_image.attrs['label']

            blur=blur_image(pil_im)
            cnt=cnt+1
            Myname=im+ " " + str(cnt)
            blur_img=f.create_dataset(Myname,data=blur)
            blur_img.attrs['font']=letter_image.attrs['font']
            blur_img.attrs['letter']=letter_image.attrs['letter']
            blur_img.attrs['label']=letter_image.attrs['label']




f.close()
db.close()

0
